In [ ]:

import os

import pandas as pd
from IPython.display import display

from tensorflow import keras

from sklearn.model_selection import train_test_split

from proteinbert import OutputType, OutputSpec, FinetuningModelGenerator, load_pretrained_model, finetune, evaluate_by_len
from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs


In [2]:
import pickle

with open('./DBP_model_param/model_attention.pt', 'rb') as f:
    model_generator = pickle.load(f)
with open('./DBP_model_param/model_input_encoder_attention.pt', 'rb') as f:
    input_encoder = pickle.load(f)
with open('./DBP_model_param/model_OUTPUT_SPEC_attention.pt', 'rb') as f:
    OUTPUT_SPEC = pickle.load(f)


In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [ ]:
test_set = pd.read_csv('./datasets/DBP_Predict_PDB.test.csv').dropna().drop_duplicates()

X_test,y_preds,y_trues,results, confusion_matrix = evaluate_by_len(model_generator, input_encoder, OUTPUT_SPEC, test_set['protein_ID'],test_set['seq'], test_set['label'], \
        start_seq_len = 512, start_batch_size = 32)

print('Test-set performance:')
display(results)

print('Confusion matrix:')
display(confusion_matrix)



In [5]:
dataframe_512 = pd.DataFrame()
dataframe_1024 = pd.DataFrame()
dataframe_2048 = pd.DataFrame()

dataframe_512['protein_ID'] = X_test[0]
dataframe_512['y_preds'] = y_preds[0]
dataframe_512['y_trues'] = y_trues[0]
dataframe_1024['protein_ID'] = X_test[1]
dataframe_1024['y_preds'] = y_preds[1]
dataframe_1024['y_trues'] = y_trues[1]
dataframe_2048['protein_ID'] = X_test[2]
dataframe_2048['y_preds'] = y_preds[2]
dataframe_2048['y_trues'] = y_trues[2]

dataframe = pd.concat([dataframe_512,dataframe_1024,dataframe_2048])
dataframe.to_csv('./PLM-DBPPred_predict_result.csv')

In [6]:
from scipy.stats import spearmanr
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix,precision_score,recall_score,f1_score
import numpy as np

y_pred = np.array(dataframe['y_preds'])
# print(y_pred)

y_true = dataframe['y_trues'].values

y_pred_classes = (y_pred >= 0.5)
y_true_classes = (y_true >= 0.5)

# print(y_pred_classes)
# print(y_true_classes)
results = pd.DataFrame()
results['AUC'] = [roc_auc_score(y_true, y_pred)]
results['Accuracy'] = [accuracy_score(y_true, y_pred_classes)]
results['precision'] = [precision_score(y_true, y_pred_classes)]
results['recall_score'] = [recall_score(y_true, y_pred_classes)]
results['f1_score'] = [f1_score(y_true, y_pred_classes)]

tn, fp, fn, tp = confusion_matrix(y_true,y_pred_classes).ravel()

results['sensitivity'] = [tp / (tp + fn)]
results['specificity'] = [tn / (tn + fp)]

confusion_matrix = pd.DataFrame(confusion_matrix(y_true,y_pred_classes))
confusion_matrix
results


,AUC,Accuracy,precision,recall_score,f1_score,sensitivity,specificity
0,0.851533,0.791667,0.866109,0.69,0.768089,0.69,0.893333
